In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 300
generations = int(3000/necoc)
generations

10

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [6]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

slice = random.sample(range(2**(26 - 1) - 1), necoc)

number_codes = number_codes[:,slice].tolist()

classes = []
for i in range(necoc):
    code_list = np.array(number_codes).T[i]
    classes.append(np.where(code_list==1)[0].tolist())

matrix_errors = []

for i in range(25):
    for j in range(i+1,26):
        matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
min_error = np.min(matrix_errors)

print(min_error)

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000
126


In [7]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/30".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/30
number of class1:653
number of class2:947

 ****** Running generation 0 ****** 

Population's average fitness: 0.50689 stdev: 0.08521
Best fitness: 0.66938 - size: (2, 3) - species 3 - id 83
Average adjusted fitness: 0.162
Mean genetic distance 2.675, standard deviation 0.494
Population of 199 members in 12 species
Total extinctions: 0
Generation time: 3.170 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.54531 stdev: 0.07465
Best fitness: 0.68937 - size: (2, 4) - species 9 - id 356
Average adjusted fitness: 0.158
Mean genetic distance 2.769, standard deviation 0.560
Population of 199 members in 15 species
Total extinctions: 0
Generation time: 3.760 sec (3.465 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.55036 stdev: 0.07482
Best fitness: 0.68937 - size: (2, 4) - species 4 - id 356
Average adjusted fitness: 0.186
Mean genetic distance 2.854, standard deviation 0.587
Population of 200 members in 20 species
Total 

Population's average fitness: 0.54433 stdev: 0.05539
Best fitness: 0.67688 - size: (3, 6) - species 8 - id 474
Average adjusted fitness: 0.111
Mean genetic distance 2.944, standard deviation 0.547
Population of 203 members in 25 species
Total extinctions: 0
Generation time: 4.287 sec (3.842 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.53874 stdev: 0.05637
Best fitness: 0.67688 - size: (3, 6) - species 8 - id 474
Average adjusted fitness: 0.094
Mean genetic distance 3.033, standard deviation 0.539
Population of 201 members in 29 species
Total extinctions: 0
Generation time: 4.605 sec (3.969 average)

 ****** Running generation 6 ****** 

Population's average fitness: 0.53832 stdev: 0.05913
Best fitness: 0.67688 - size: (3, 6) - species 8 - id 474
Average adjusted fitness: 0.162
Mean genetic distance 3.113, standard deviation 0.526
Population of 202 members in 30 species
Total extinctions: 0
Generation time: 4.562 sec (4.054 average)

 ****** Running ge

Population's average fitness: 0.56545 stdev: 0.07882
Best fitness: 0.65750 - size: (2, 4) - species 5 - id 875
Average adjusted fitness: 0.185
Mean genetic distance 3.042, standard deviation 0.509
Population of 200 members in 36 species
Total extinctions: 0
Generation time: 4.252 sec (3.714 average)

 ****** Running generation 9 ****** 

Population's average fitness: 0.57425 stdev: 0.07388
Best fitness: 0.65750 - size: (2, 4) - species 5 - id 875
Average adjusted fitness: 0.187
Mean genetic distance 3.035, standard deviation 0.514
Population of 199 members in 36 species
Total extinctions: 0
Generation time: 4.317 sec (3.775 average)
running:5/30
number of class1:844
number of class2:756

 ****** Running generation 0 ****** 

Population's average fitness: 0.50150 stdev: 0.03670
Best fitness: 0.58813 - size: (2, 3) - species 3 - id 14
Average adjusted fitness: 0.160
Mean genetic distance 2.701, standard deviation 0.506
Population of 199 members in 15 species
Total extinctions: 0
Generati

Population's average fitness: 0.52985 stdev: 0.05807
Best fitness: 0.72375 - size: (3, 5) - species 6 - id 536
Average adjusted fitness: 0.129
Mean genetic distance 2.885, standard deviation 0.559
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 3.635 sec (3.250 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.52540 stdev: 0.06767
Best fitness: 0.72375 - size: (3, 5) - species 5 - id 536
Average adjusted fitness: 0.171
Mean genetic distance 2.916, standard deviation 0.575
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 4.046 sec (3.449 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.53182 stdev: 0.06488
Best fitness: 0.72375 - size: (3, 5) - species 6 - id 536
Average adjusted fitness: 0.197
Mean genetic distance 2.949, standard deviation 0.563
Population of 201 members in 24 species
Total extinctions: 0
Generation time: 4.010 sec (3.561 average)

 ****** Running ge

Population's average fitness: 0.51804 stdev: 0.02741
Best fitness: 0.58375 - size: (2, 3) - species 14 - id 558
Average adjusted fitness: 0.053
Mean genetic distance 2.920, standard deviation 0.545
Population of 204 members in 26 species
Total extinctions: 0
Generation time: 4.146 sec (3.663 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.51846 stdev: 0.02955
Best fitness: 0.58375 - size: (2, 3) - species 5 - id 558
Average adjusted fitness: 0.059
Mean genetic distance 2.890, standard deviation 0.549
Population of 199 members in 26 species
Total extinctions: 0
Generation time: 4.117 sec (3.720 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.51915 stdev: 0.02875
Best fitness: 0.58375 - size: (2, 3) - species 5 - id 558
Average adjusted fitness: 0.065
Mean genetic distance 2.922, standard deviation 0.521
Population of 200 members in 26 species
Total extinctions: 0
Generation time: 4.129 sec (3.765 average)

 ****** Running g

...

Population's average fitness: 0.49641 stdev: 0.11239
Best fitness: 0.65625 - size: (2, 3) - species 1 - id 4
Average adjusted fitness: 0.170
Mean genetic distance 2.688, standard deviation 0.470
Population of 199 members in 18 species
Total extinctions: 0
Generation time: 2.796 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.56420 stdev: 0.09596
Best fitness: 0.66125 - size: (2, 3) - species 7 - id 351
Average adjusted fitness: 0.199
Mean genetic distance 2.835, standard deviation 0.508
Population of 200 members in 21 species
Total extinctions: 0
Generation time: 3.166 sec (2.981 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.56480 stdev: 0.09698
Best fitness: 0.66125 - size: (2, 3) - species 16 - id 351
Average adjusted fitness: 0.233
Mean genetic distance 2.846, standard deviation 0.549
Population of 199 members in 21 species
Total extinctions: 0
Generation time: 3.447 sec (3.136 average)

 ****** Running generation 3 ******

Population's average fitness: 0.57560 stdev: 0.12576
Best fitness: 0.72875 - size: (3, 5) - species 5 - id 462
Average adjusted fitness: 0.246
Mean genetic distance 2.966, standard deviation 0.576
Population of 204 members in 25 species
Total extinctions: 0
Generation time: 3.650 sec (3.313 average)

 ****** Running generation 5 ****** 

Population's average fitness: 0.57448 stdev: 0.12602
Best fitness: 0.72875 - size: (3, 5) - species 5 - id 462
Average adjusted fitness: 0.233
Mean genetic distance 3.004, standard deviation 0.580
Population of 206 members in 25 species
Total extinctions: 0
Generation time: 3.896 sec (3.411 average)

 ****** Running generation 6 ****** 

Population's average fitness: 0.57601 stdev: 0.12703
Best fitness: 0.72875 - size: (3, 5) - species 1 - id 462
Average adjusted fitness: 0.230
Mean genetic distance 3.019, standard deviation 0.562
Population of 203 members in 26 species
Total extinctions: 0
Generation time: 3.991 sec (3.493 average)

 ****** Running ge

Population's average fitness: 0.52621 stdev: 0.04028
Best fitness: 0.61813 - size: (3, 7) - species 6 - id 1454
Average adjusted fitness: 0.071
Mean genetic distance 2.941, standard deviation 0.539
Population of 197 members in 29 species
Total extinctions: 0
Generation time: 3.765 sec (3.505 average)

 ****** Running generation 9 ****** 

Population's average fitness: 0.52466 stdev: 0.04047
Best fitness: 0.61813 - size: (3, 7) - species 1 - id 1454
Average adjusted fitness: 0.083
Mean genetic distance 2.944, standard deviation 0.532
Population of 203 members in 29 species
Total extinctions: 0
Generation time: 3.856 sec (3.540 average)
running:293/30
number of class1:829
number of class2:771

 ****** Running generation 0 ****** 

Population's average fitness: 0.49868 stdev: 0.03275
Best fitness: 0.61438 - size: (2, 3) - species 4 - id 8
Average adjusted fitness: 0.108
Mean genetic distance 2.672, standard deviation 0.489
Population of 200 members in 13 species
Total extinctions: 0
Gener

Population's average fitness: 0.54000 stdev: 0.09712
Best fitness: 0.68000 - size: (4, 5) - species 27 - id 549
Average adjusted fitness: 0.209
Mean genetic distance 3.012, standard deviation 0.523
Population of 201 members in 32 species
Total extinctions: 0
Generation time: 4.051 sec (3.592 average)

 ****** Running generation 3 ****** 

Population's average fitness: 0.55219 stdev: 0.09126
Best fitness: 0.68000 - size: (4, 5) - species 27 - id 549
Average adjusted fitness: 0.188
Mean genetic distance 3.063, standard deviation 0.512
Population of 204 members in 34 species
Total extinctions: 0
Generation time: 5.988 sec (4.191 average)

 ****** Running generation 4 ****** 

Population's average fitness: 0.57209 stdev: 0.08141
Best fitness: 0.68000 - size: (4, 5) - species 27 - id 549
Average adjusted fitness: 0.201
Mean genetic distance 3.070, standard deviation 0.512
Population of 202 members in 34 species
Total extinctions: 0
Generation time: 4.699 sec (4.292 average)

 ****** Running

Population's average fitness: 0.55957 stdev: 0.08061
Best fitness: 0.66250 - size: (3, 4) - species 10 - id 699
Average adjusted fitness: 0.152
Mean genetic distance 2.876, standard deviation 0.523
Population of 200 members in 23 species
Total extinctions: 0
Generation time: 4.669 sec (4.130 average)

 ****** Running generation 7 ****** 

Population's average fitness: 0.56724 stdev: 0.07801
Best fitness: 0.66250 - size: (3, 4) - species 10 - id 699
Average adjusted fitness: 0.171
Mean genetic distance 2.922, standard deviation 0.506
Population of 203 members in 23 species
Total extinctions: 0
Generation time: 5.260 sec (4.271 average)

 ****** Running generation 8 ****** 

Population's average fitness: 0.56035 stdev: 0.08394
Best fitness: 0.66250 - size: (3, 4) - species 10 - id 699
Average adjusted fitness: 0.184
Mean genetic distance 2.965, standard deviation 0.533
Population of 200 members in 24 species
Total extinctions: 0
Generation time: 4.984 sec (4.350 average)

 ****** Running

In [8]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.5075

In [9]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.555
Avg Base Learner:0.6670041666666666


In [10]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:2453 Total connections::3302
